In [1]:
import requests
from bs4 import BeautifulSoup
import random
import pandas as pd
import re
import numpy as np
import time
import jupytext

In [2]:
Countries_1stLeagueName = ['premier-league', 'bundesliga', 'serie-a', 'ligue-1', 'laliga']

In [3]:
table_urls = ['https://www.transfermarkt.com/premier-league/tabelle/wettbewerb/GB1?saison_id=2021', 
              'https://www.transfermarkt.com/bundesliga/tabelle/wettbewerb/GR1?saison_id=2021',
              'https://www.transfermarkt.com/serie-a/tabelle/wettbewerb/IT1?saison_id=2021',
              'https://www.transfermarkt.com/ligue-1/tabelle/wettbewerb/FR1?saison_id=2021',
              'https://www.transfermarkt.com/laliga/tabelle/wettbewerb/ES1?saison_id=2021']

In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36',
           "Accept-Language": "en-US,en;q=0.5"}

In [12]:
responses = []
for url in table_urls:
    responses.append(requests.get(url=url, headers=headers))
    time.sleep(random.random())

In [14]:
soups = [BeautifulSoup(page.content, 'html.parser') for page in responses]

In [106]:
results = []
for soup in soups:
    table = soup.select('div.responsive-table')[0]
    teams = table.select('td.no-border-links.hauptlink')
    titles = [team.find('a').get('title') for team in teams]
    ids = [team.find('a').get('href').split('verein/')[1].split('/')[0] for team in teams]
    rows = table.select('tr')[1:]
    game_counts = [row.select('td.zentriert')[1].get_text() for row in rows]
    points = [row.select('td.zentriert')[7].get_text() for row in rows]

    for i in range(len(teams)):
        results.append({'team_title':titles[i], 'team_id':int(ids[i]), 'game_count':int(game_counts[i]), 'points':int(points[i])})


In [107]:
df = pd.DataFrame(results)

In [108]:
uefa_paricipitants_url = 'https://www.transfermarkt.com/uefa-champions-league/teilnehmer/pokalwettbewerb/CL/saison_id/2021'

In [84]:
uefa_response = requests.get(url=uefa_paricipitants_url, headers=headers)

In [85]:
uefa_soup = BeautifulSoup(uefa_response.content, 'html.parser')

In [87]:
uefa_tags = uefa_soup.select('td.links.no-border-links.hauptlink a')

In [88]:
len(uefa_tags)

32

In [118]:
uefa_titles = [tag.get('title') for tag in uefa_tags]
uefa_ids = [int(tag.get('href').split('verein/')[1].split('/')[0]) for tag in uefa_tags]

In [109]:
df

,team_title,team_id,game_count,points
0,Manchester City,281,38,93
1,Liverpool FC,31,38,92
2,Chelsea FC,631,38,74
3,Tottenham Hotspur,148,38,71
4,Arsenal FC,11,38,69
...,...,...,...,...
89,RCD Mallorca,237,38,39
90,Cádiz CF,2687,38,39
91,Granada CF,16795,38,38
92,Levante UD,3368,38,35


In [122]:
df['uefa_status'] = df.team_id.isin(uefa_ids)
df['team_performance'] = df.points/df.game_count

In [123]:
df

,team_title,team_id,game_count,points,uefa_status,team_performance
0,Manchester City,281,38,93,True,2.447368
1,Liverpool FC,31,38,92,True,2.421053
2,Chelsea FC,631,38,74,True,1.947368
3,Tottenham Hotspur,148,38,71,False,1.868421
4,Arsenal FC,11,38,69,False,1.815789
...,...,...,...,...,...,...
89,RCD Mallorca,237,38,39,False,1.026316
90,Cádiz CF,2687,38,39,False,1.026316
91,Granada CF,16795,38,38,False,1.000000
92,Levante UD,3368,38,35,False,0.921053


In [130]:
df.to_csv('tables.csv')